# 1. Data Collection

# Problem Statement

Due to covid and the presence of lockdowns which has affected public housing project delays in Singapore, HDB resale prices has been steadily increasing and has become a common discussion topics especially amongst first time young buyers.  

I am curious to find out what features influence HDB resale prices and help potential buyers find out if the current asking prices of HDBs are reasonable by using regression models to predict HDB prices.

The increasing cost of living comes to mind for young Singaporeans looking to purchase a home and start a family. This model would serve a guide for them as part of their home purchase decision making process.

Regression models for consideration: Linear, Lasso, Ridge, Random Forest, XGBoost, Neural Networks

Success Metrics: Model performance will be guided by RMSE. We will seek to find the best performing model based on the lowest RMSE score. 

# Imports

In [1]:
import numpy as np
import pandas as pd
import requests

# Reading in HDB Data Sources

Data received from https://data.gov.sg/dataset/resale-flat-prices with HDB resale transactions dating back to 1990. Most recent data is Oct 2021.

In [2]:
pd.set_option('display.max_rows', 1000)

In [3]:
hdb_2000_2012feb = pd.read_csv('../datasets/original-hdb-data/resale-flat-prices-based-on-approval-date-2000-feb-2012.csv')
hdb_2012mar_2014dec = pd.read_csv('../datasets/original-hdb-data/resale-flat-prices-based-on-registration-date-from-mar-2012-to-dec-2014.csv')
hdb_2015_2016 = pd.read_csv('../datasets/original-hdb-data/resale-flat-prices-based-on-registration-date-from-jan-2015-to-dec-2016.csv')
hdb_2017_to_recent = pd.read_csv('../datasets/original-hdb-data/resale-flat-prices-based-on-registration-date-from-jan-2017-onwards.csv')

In [4]:
# briefly examine df
hdb_2017_to_recent.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 04 months,232000.0
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,60 years 07 months,250000.0
2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,262000.0
3,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980,62 years 01 month,265000.0
4,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,265000.0


In [5]:
# checking data types
hdb_2017_to_recent.dtypes

month                   object
town                    object
flat_type               object
block                   object
street_name             object
storey_range            object
floor_area_sqm         float64
flat_model              object
lease_commence_date      int64
remaining_lease         object
resale_price           float64
dtype: object

In [6]:
hdb_2017_to_recent['flat_model'].unique()

array(['Improved', 'New Generation', 'DBSS', 'Standard', 'Apartment',
       'Simplified', 'Model A', 'Premium Apartment', 'Adjoined flat',
       'Model A-Maisonette', 'Maisonette', 'Type S1', 'Type S2',
       'Model A2', 'Terrace', 'Improved-Maisonette', 'Premium Maisonette',
       'Multi Generation', 'Premium Apartment Loft', '2-room'],
      dtype=object)

In [7]:
hdb_2017_to_recent['flat_type'].unique()

array(['2 ROOM', '3 ROOM', '4 ROOM', '5 ROOM', 'EXECUTIVE', '1 ROOM',
       'MULTI-GENERATION'], dtype=object)

# Collecting Longitudes and Latitudes of HDB transactions

- Create longitude and latitude columns

Real estate prices are closely driven by location. We will create longitude and latitude data from the block and street_name columns to get their exact locations by calling the government's OneMap API. 

In [24]:
# create function to add in new columns to df
def add_long_lat(df):
    
    '''add longitude and latitude columns to the df'''
    
    df['longitude'] = ''
    df['latitude'] = ''
    return df

In [25]:
# function to get lat and long of the hdb using OneMap API
def get_coordinate(df, df_name):
    
    '''get HDB coordinate by calling the OneMap API through searching with block and street name data'''
    
    row = 0        

    for block, street in zip(df['block'], df['street_name']):  

        if df['longitude'][row] != '':               # skip row if coordinates already filled
            print(f'Row {row} already filled')       
            row += 1                                 # shift to check next row 

        else:

            url = f'https://developers.onemap.sg/commonapi/search?searchVal={block} {street}&returnGeom=Y&getAddrDetails=Y'
            res = requests.get(url)

            # Check Status Code
            if res.status_code != 200:
                print("Error",res.status_code)
                break

            else:

                print(f'{row} Retrieved coordinates from {url}')
                data = res.json()

                try:
                    longitude = data['results'][0]['LONGITUDE']
                    latitude = data['results'][0]['LATITUDE']

                # if no data, we will return empty strings
                except:
                    longitude = ''
                    latitude = ''

                df['longitude'][row] = longitude
                df['latitude'][row] = latitude

                row += 1
                df.to_csv(f'{df_name}.csv', index=False)


    return df

In [ ]:
# add_long_lat(hdb_2017_to_recent)

In [ ]:
#get_coordinate(hdb_2017_to_recent, 'hdb_2017_recent_long_lat')

In [ ]:
#add_long_lat(hdb_2015_2016)

In [ ]:
#get_coordinate(hdb_2015_2016, 'hdb_2015_2016_long_lat')

In [ ]:
#add_long_lat(hdb_2012mar_2014dec)

In [ ]:
#get_coordinate(hdb_2012mar_2014dec, 'hdb_2012mar_2014dec_long_lat')

In [9]:
# extracting data from oct 2011 to 2012 feb
# this will complete the entire data collection of 10 years worth of HDB transaction

hdb_2011_oct = hdb_2000_2012feb[hdb_2000_2012feb['month'].str.contains('2011-10')]
hdb_2011_nov = hdb_2000_2012feb[hdb_2000_2012feb['month'].str.contains('2011-11')]
hdb_2011_dec = hdb_2000_2012feb[hdb_2000_2012feb['month'].str.contains('2011-12')]
hdb_2012_to_feb = hdb_2000_2012feb[hdb_2000_2012feb['month'].str.contains('2012')]

In [10]:
hdb_2011_oct_to_2012_feb = pd.concat([hdb_2011_oct, hdb_2011_nov, hdb_2011_dec, hdb_2012_to_feb])
hdb_2011_oct_to_2012_feb.shape

(8120, 12)

In [ ]:
#add_long_lat(hdb_2011_oct_to_2012_feb)

In [ ]:
#hdb_2011_oct_to_2012_feb.index = np.arange(len(hdb_2011_oct_to_2012_feb))

In [ ]:
#get_coordinate(hdb_2011_oct_to_2012_feb, 'hdb_2011_oct_to_2012_feb_long_lat')

# Collecting Longitudes and latitudes of Amenities 

In [17]:
# create function to pull long, lat of amenities
def get_amenities(amenity_search, num_page, amenity_df, amenity_col_name):
    ls = []

    for page in range(1,num_page+1):

        try:

            url = f'https://developers.onemap.sg/commonapi/search?searchVal={amenity_search}&returnGeom=Y&getAddrDetails=Y&pageNum={page}'
            res = requests.get(url)

            # Check Status Code
            if res.status_code != 200:
                print("Error",res.status_code)

            else:
                print(f'Retrieved page {page} for {amenity_search} geo coordinates')
                data = res.json()
                for i in range(10):
                    results = {amenity_col_name: data['results'][i]['BUILDING'],
                           'longitude': data['results'][i]['LONGITUDE'],
                           'latitude': data['results'][i]['LATITUDE']}
                    ls.append(results)

                page += 1
                
                

        except:
            pass
    
    amenity_df = pd.DataFrame(ls)
    amenity_df.to_csv(f'../datasets/amenities/{amenity_col_name}.csv', index=False)
    
    return amenity_df 

## Scraping MRT station coordinates

In [37]:
mrt = get_amenities('mrt station', 100, 'mrt_stn', 'mrt_station')
mrt

Retrieved Page1 for mrt station Coordinates
Retrieved Page2 for mrt station Coordinates
Retrieved Page3 for mrt station Coordinates
Retrieved Page4 for mrt station Coordinates
Retrieved Page5 for mrt station Coordinates
Retrieved Page6 for mrt station Coordinates
Retrieved Page7 for mrt station Coordinates
Retrieved Page8 for mrt station Coordinates
Retrieved Page9 for mrt station Coordinates
Retrieved Page10 for mrt station Coordinates
Retrieved Page11 for mrt station Coordinates
Retrieved Page12 for mrt station Coordinates
Retrieved Page13 for mrt station Coordinates
Retrieved Page14 for mrt station Coordinates
Retrieved Page15 for mrt station Coordinates
Retrieved Page16 for mrt station Coordinates
Retrieved Page17 for mrt station Coordinates
Retrieved Page18 for mrt station Coordinates
Retrieved Page19 for mrt station Coordinates
Retrieved Page20 for mrt station Coordinates
Retrieved Page21 for mrt station Coordinates
Retrieved Page22 for mrt station Coordinates
Retrieved Page23 fo

,mrt station,longitude,latitude
0,CITY HALL MRT STATION (EW13 / NS25),103.852585580366,1.29293672227779
1,BAYFRONT MRT STATION (DT16 / CE1),103.859079764874,1.28187378879209
2,SUNGEI BEDOK MRT STATION (TE31 / DT37),103.957183823478,1.32040148729112
3,HARBOURFRONT MRT STATION (NE1 / CC29),103.821530157095,1.26538938374901
4,JURONG EAST MRT STATION (EW24 / NS1),103.742286332403,1.33315281585758
5,RAFFLES PLACE MRT STATION (EW14 / NS26),103.851461712342,1.28412561043658
6,CALDECOTT MRT STATION (TE9),103.840161782075,1.33724561271722
7,CASHEW MRT STATION (DT2),103.76443921414,1.36981544925552
8,LENTOR MRT STATION (TE5),103.835743809669,1.38550657972169
9,SOMERSET MRT STATION (NS23),103.839085753124,1.30026416739006


There are several mrt stations with names starting with 'UOB', 'OCBC' or 'DBS', we will drop those columns.

In [38]:
mrt = mrt[~mrt['mrt_station'].str.contains('UOB|DBS|OCBC|DEPOT')]

In [39]:
mrt['mrt_station'] = mrt['mrt_station'].str.replace('MRT STATION.*', '')
mrt

C:\Users\caspe\anaconda3\envs\capstone\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.
C:\Users\caspe\anaconda3\envs\capstone\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,mrt station,longitude,latitude
0,CITY HALL,103.852585580366,1.29293672227779
1,BAYFRONT,103.859079764874,1.28187378879209
2,SUNGEI BEDOK,103.957183823478,1.32040148729112
3,HARBOURFRONT,103.821530157095,1.26538938374901
4,JURONG EAST,103.742286332403,1.33315281585758
5,RAFFLES PLACE,103.851461712342,1.28412561043658
6,CALDECOTT,103.840161782075,1.33724561271722
7,CASHEW,103.76443921414,1.36981544925552
8,LENTOR,103.835743809669,1.38550657972169
9,SOMERSET,103.839085753124,1.30026416739006


In [40]:
mrt.drop_duplicates(subset='mrt_station', inplace=True)

C:\Users\caspe\anaconda3\envs\capstone\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [46]:
mrt = mrt.sort_values(by='mrt station')

In [47]:
mrt.shape

(155, 3)

In [48]:
mrt['longitude'] = mrt['longitude'].astype(float)
mrt['latitude'] = mrt['latitude'].astype(float)

In [49]:
mrt.index = np.arange(len(mrt))

In [50]:
mrt.to_csv('../datasets/amenities/mrt_cleaned.csv', index=False)

## Scraping Pri Sch coordinates

In [30]:
pri_sch = get_amenities('primary school', 25, 'df_pri_sch', 'primary_school')

Retrieved Page1 for primary school Coordinates
Retrieved Page2 for primary school Coordinates
Retrieved Page3 for primary school Coordinates
Retrieved Page4 for primary school Coordinates
Retrieved Page5 for primary school Coordinates
Retrieved Page6 for primary school Coordinates
Retrieved Page7 for primary school Coordinates
Retrieved Page8 for primary school Coordinates
Retrieved Page9 for primary school Coordinates
Retrieved Page10 for primary school Coordinates
Retrieved Page11 for primary school Coordinates
Retrieved Page12 for primary school Coordinates
Retrieved Page13 for primary school Coordinates
Retrieved Page14 for primary school Coordinates
Retrieved Page15 for primary school Coordinates
Retrieved Page16 for primary school Coordinates
Retrieved Page17 for primary school Coordinates
Retrieved Page18 for primary school Coordinates
Retrieved Page19 for primary school Coordinates
Retrieved Page20 for primary school Coordinates
Retrieved Page21 for primary school Coordinates
R

,primary school,longitude,latitude
0,ZHONGHUA PRIMARY SCHOOL,103.869762438084,1.36020329705047
1,ZHENGHUA PRIMARY SCHOOL,103.769313521752,1.37954887512229
2,ZHANGDE PRIMARY SCHOOL,103.825951875662,1.28421153335379
3,YUMIN PRIMARY SCHOOL,103.950461927088,1.35129177656981
4,YUHUA PRIMARY SCHOOL,103.741105772644,1.34280230475033
5,YU NENG PRIMARY SCHOOL,103.932144857355,1.33388930344069
6,JIEMIN PRIMARY SCHOOL,103.830425490093,1.42768847665227
7,XISHAN PRIMARY SCHOOL,103.837755319448,1.43347186925064
8,AHMAD IBRAHIM PRIMARY SCHOOL,103.832942401086,1.43315271543517
9,YISHUN PRIMARY SCHOOL,103.834087115915,1.43338602975674


In [ ]:
pri_sch['primary_school'] = pri_sch['primary_school'].str.replace('(', '')
pri_sch['primary_school'] = pri_sch['primary_school'].str.replace(')', '')
pri_sch['primary_school'] = pri_sch['primary_school'].str.replace('AFTERSCHOOL @ ', '')
pri_sch['primary_school'] = pri_sch['primary_school'].str.replace('AMP-MERCU STUDENT CARE CENTRE @ ', '')
pri_sch['primary_school'] = pri_sch['primary_school'].str.replace('BIG HEART STUDENT CARE ', '')
pri_sch['primary_school'] = pri_sch['primary_school'].str.replace('COMMIT LEARNING SCHOOLHOUSE @ ', '')
pri_sch['primary_school'] = pri_sch['primary_school'].str.replace('KNOWLEDGE PARK EDUCARE PTE LTD ', '')
pri_sch['primary_school'] = pri_sch['primary_school'].str.replace('LEARNING LEAP PTE LTD ', '')
pri_sch['primary_school'] = pri_sch['primary_school'].str.replace('LEARNING STUDIO BY ZENITANT ', '')
pri_sch['primary_school'] = pri_sch['primary_school'].str.replace('LEARNING STUDIO EDUCARE @ ', '')
pri_sch['primary_school'] = pri_sch['primary_school'].str.replace('MERCU STUDENT CARE CENTRE @ ', '')
pri_sch['primary_school'] = pri_sch['primary_school'].str.replace('MOE KINDERGARTEN @ ', '')
pri_sch['primary_school'] = pri_sch['primary_school'].str.replace('POWERLIFE STUDENT SERVICES @ ', '')
pri_sch['primary_school'] = pri_sch['primary_school'].str.replace('SHG STUDENT CARE ', '')
pri_sch['primary_school'] = pri_sch['primary_school'].str.replace('THE LEARNING HARBOUR @ ', '')
pri_sch['primary_school'] = pri_sch['primary_school'].str.replace('TRI-LOVE STUDENT CARE @ ', '')
pri_sch['primary_school'] = pri_sch['primary_school'].str.replace('THE LEARNING HARBOUR @ ', '')
pri_sch['primary_school'] = pri_sch['primary_school'].str.replace('WHEE!CANOPUS STUDENT CARE @ ', '')
pri_sch['primary_school'] = pri_sch['primary_school'].str.replace('YMCA STUDENT CARE CENTRE @ ', '')

In [ ]:
pri_sch.drop_duplicates(subset='primary_school', inplace=True)

In [ ]:
pri_sch.to_csv('../datasets/amenities/pri_sch_cleaned.csv', index=False)

## Scraping Sec Sch coordinates

In [31]:
get_amenities('secondary school', 15, 'df_sec_sch', 'secondary_school')

Retrieved Page1 for secondary school Coordinates
Retrieved Page2 for secondary school Coordinates
Retrieved Page3 for secondary school Coordinates
Retrieved Page4 for secondary school Coordinates
Retrieved Page5 for secondary school Coordinates
Retrieved Page6 for secondary school Coordinates
Retrieved Page7 for secondary school Coordinates
Retrieved Page8 for secondary school Coordinates
Retrieved Page9 for secondary school Coordinates
Retrieved Page10 for secondary school Coordinates
Retrieved Page11 for secondary school Coordinates
Retrieved Page12 for secondary school Coordinates
Retrieved Page13 for secondary school Coordinates
Retrieved Page14 for secondary school Coordinates
Retrieved Page15 for secondary school Coordinates


,secondary school,longitude,latitude
0,ZHONGHUA SECONDARY SCHOOL,103.869430481414,1.34840720846208
1,ZHENGHUA SECONDARY SCHOOL,103.765510638527,1.38836583415352
2,YUYING SECONDARY SCHOOL,103.890194755287,1.35713595302467
3,YUSOF ISHAK SECONDARY SCHOOL,103.760032628302,1.34233921440623
4,YUHUA SECONDARY SCHOOL,103.722687235839,1.34718626876968
5,YUAN CHING SECONDARY SCHOOL,103.72089870437,1.34200635764467
6,JURONG SECONDARY SCHOOL,103.723952961648,1.33002032220942
7,OUTRAM SECONDARY SCHOOL,103.837127264403,1.28651357464735
8,YISHUN TOWN SECONDARY SCHOOL,103.838094226361,1.43232156717386
9,ORCHID PARK SECONDARY SCHOOL,103.837859241343,1.41532032779634


## Scraping Mall coordinates

The OneMap API doesn't seem to return all the shopping malls in Singapore when calling it. Instead, we will scrape the names of the shopping malls from [Wikipedia](https://en.wikipedia.org/wiki/List_of_shopping_malls_in_Singapore) using BeautifulSoup. The Wiki link has a list of 171 shopping malls as of August 2020.

In [12]:
from bs4 import BeautifulSoup

In [13]:
# get url
url = 'https://en.wikipedia.org/wiki/List_of_shopping_malls_in_Singapore'
res = requests.get(url)

# create soup object
soup = BeautifulSoup(res.content)

# create a list to append text data
ls = []

for object in soup.find_all('ul'):
    ls.append(object.text)
    
# all the shopping mall names are in element positions from 1 to 7
# we will extract by slicing it

ls = ls[1:8]

# create mall list to append all the shopping malls names
malls = []

for elem in ls:
    names = elem.split('\n')
    malls.extend(names)
    
malls

['100 AM',
 '313@Somerset',
 'Aperia',
 'Balestier Hill Shopping Centre',
 'Bugis Cube',
 'Bugis Junction',
 'Bugis+',
 'Capitol Piazza',
 'Cathay Cineleisure Orchard',
 'Clarke Quay Central',
 'The Centrepoint',
 'City Square Mall',
 'City Gate Mall',
 'CityLink Mall',
 'Duo',
 'Far East Plaza',
 'Funan',
 'Great World City',
 'HDB Hub',
 'Holland Village Shopping Mall',
 'ION Orchard',
 'Junction 8',
 'Knightsbridge[1]',
 'Liat Towers',
 'Lucky Plaza',
 'Marina Bay Sands',
 'The Shoppes at Marina Bay Sands',
 'Marina Bay Financial Centre Tower 3',
 'Marina Bay Link Mall',
 'Marina One',
 'Marina Square',
 'Millenia Walk',
 'Mustafa Shopping Centre',
 'Ngee Ann City',
 'Orchard Central',
 'Orchard Gateway',
 'Orchard Plaza',
 'Midpoint Orchard',
 'Palais Renaissance',
 "People's Park Centre",
 "People's Park Complex",
 'Plaza Singapura',
 'PoMo',
 'Raffles City',
 'Scotts Square',
 'Serangoon Plaza',
 'Shaw House and Centre',
 'Sim Lim Square',
 'Singapore Shopping Centre',
 'The Sout

In [14]:
len(malls)

171

With our list of malls, we can call on the OneMap API to search for the geo coordinates. During our initial search, we noted that some of the malls did not return longitudes and latitudes as their names were misspelt. We will clean them up and search again afterwards. 

To correct the following spellings of the malls:

- Knightsbridge[1] to: Knightsbridge
- The Poiz[2] to: The Poiz
- Velocity@Novena Square to: Velocity @ Novena Square
- Singapore Post Centre (Singpost Centre) to: Singpost Centre
- Paya Lebar Quarter (PLQ) to: PLQ
- KINEX (formerly OneKM) to: KINEX

In [15]:
# a quick way to find and replace elements in a list, referenced from: 
# https://stackoverflow.com/questions/2582138/finding-and-replacing-elements-in-a-list

replace = {'Knightsbridge[1]': 'Knightsbridge', 'The Poiz[2]': 'The Poiz',
           'Velocity@Novena Square': 'Velocity @ Novena Square', 
           'Singapore Post Centre (Singpost Centre)': 'Singpost Centre',
           'Paya Lebar Quarter (PLQ)': 'PLQ', 'KINEX (formerly OneKM)': 'KINEX'}

replacer = replace.get
malls = [replacer(x, x) for x in malls]

In [16]:
# pulling long, lat of shopping malls 

count = 1
ls = []

for mall in malls:
    
    url = f'https://developers.onemap.sg/commonapi/search?searchVal={mall}&returnGeom=Y&getAddrDetails=Y'
    print(url)
    res = requests.get(url)

    # Check Status Code
    if res.status_code != 200:
        print("Error",res.status_code)
        
    else:
        print(f'Retrieving mall {count} geo coordinates')
        data = res.json()
        
        try:
            results = {'mall': data['results'][0]['BUILDING'],
                       'longitude': data['results'][0]['LONGITUDE'],
                       'latitude': data['results'][0]['LATITUDE']}
            
        except:
            results = {'mall': mall,
                       'longitude': 'NA',
                       'latitude': 'NA'}

        
    ls.append(results)
    count += 1


mall_df = pd.DataFrame(ls)
mall_df.to_csv(f'../datasets/amenities/mall.csv', index=False) 

https://developers.onemap.sg/commonapi/search?searchVal=100 AM&returnGeom=Y&getAddrDetails=Y
Retrieving mall 1 geo coordinates
https://developers.onemap.sg/commonapi/search?searchVal=313@Somerset&returnGeom=Y&getAddrDetails=Y
Retrieving mall 2 geo coordinates
https://developers.onemap.sg/commonapi/search?searchVal=Aperia&returnGeom=Y&getAddrDetails=Y
Retrieving mall 3 geo coordinates
https://developers.onemap.sg/commonapi/search?searchVal=Balestier Hill Shopping Centre&returnGeom=Y&getAddrDetails=Y
Retrieving mall 4 geo coordinates
https://developers.onemap.sg/commonapi/search?searchVal=Bugis Cube&returnGeom=Y&getAddrDetails=Y
Retrieving mall 5 geo coordinates
https://developers.onemap.sg/commonapi/search?searchVal=Bugis Junction&returnGeom=Y&getAddrDetails=Y
Retrieving mall 6 geo coordinates
https://developers.onemap.sg/commonapi/search?searchVal=Bugis+&returnGeom=Y&getAddrDetails=Y
Retrieving mall 7 geo coordinates
https://developers.onemap.sg/commonapi/search?searchVal=Capitol Piazz

Retrieving mall 61 geo coordinates
https://developers.onemap.sg/commonapi/search?searchVal=Thomson Plaza&returnGeom=Y&getAddrDetails=Y
Retrieving mall 62 geo coordinates
https://developers.onemap.sg/commonapi/search?searchVal=United Square&returnGeom=Y&getAddrDetails=Y
Retrieving mall 63 geo coordinates
https://developers.onemap.sg/commonapi/search?searchVal=Velocity @ Novena Square&returnGeom=Y&getAddrDetails=Y
Retrieving mall 64 geo coordinates
https://developers.onemap.sg/commonapi/search?searchVal=Wheelock Place&returnGeom=Y&getAddrDetails=Y
Retrieving mall 65 geo coordinates
https://developers.onemap.sg/commonapi/search?searchVal=Wisma Atria&returnGeom=Y&getAddrDetails=Y
Retrieving mall 66 geo coordinates
https://developers.onemap.sg/commonapi/search?searchVal=Zhongshan Mall&returnGeom=Y&getAddrDetails=Y
Retrieving mall 67 geo coordinates
https://developers.onemap.sg/commonapi/search?searchVal=Century Square&returnGeom=Y&getAddrDetails=Y
Retrieving mall 68 geo coordinates
https://

Retrieving mall 124 geo coordinates
https://developers.onemap.sg/commonapi/search?searchVal=Rivervale Plaza&returnGeom=Y&getAddrDetails=Y
Retrieving mall 125 geo coordinates
https://developers.onemap.sg/commonapi/search?searchVal=The Seletar Mall&returnGeom=Y&getAddrDetails=Y
Retrieving mall 126 geo coordinates
https://developers.onemap.sg/commonapi/search?searchVal=Buangkok Square&returnGeom=Y&getAddrDetails=Y
Retrieving mall 127 geo coordinates
https://developers.onemap.sg/commonapi/search?searchVal=The Midtown&returnGeom=Y&getAddrDetails=Y
Retrieving mall 128 geo coordinates
https://developers.onemap.sg/commonapi/search?searchVal=Upper Serangoon Shopping Centre&returnGeom=Y&getAddrDetails=Y
Retrieving mall 129 geo coordinates
https://developers.onemap.sg/commonapi/search?searchVal=Beauty World Centre&returnGeom=Y&getAddrDetails=Y
Retrieving mall 130 geo coordinates
https://developers.onemap.sg/commonapi/search?searchVal=Beauty World Plaza&returnGeom=Y&getAddrDetails=Y
Retrieving mal